# Dynamic UPF Placement and Chaining Reconfiguration (UPCR)

https://www.sciencedirect.com/science/article/pii/S1389128622002900

In [1]:
import pickle
import math
import time

# PYOMO AND SOLVER 
from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
from pyomo.core import Var

# FIle with model Parameters
import Parameters as params

In [2]:
def import_data(dir_data, file_name):
    """THis function imports the data that it is in dir_data+file_name"""
    with open (dir_data+file_name, "rb") as fp:
        return pickle.load(fp)

In [3]:
def export_data(file_name, file_data):
    """Esta funcion me permite salvar en un fichero con nombbre file_data la información contenida en file_data"""
    with open(file_name, "wb") as fp:
        pickle.dump(file_data, fp)
    return

## Importing Dta


In [4]:
dir_data = params.dir_data

In [5]:
# Session information
S = import_data(dir_data, "S") 

Dict_Tsfc_s = import_data(dir_data, "Dict_Tsfc_s"+str(S))           # Matrix with VNFs in SFC: 1 if VNF f ∈ Fs in SFCR s ∈ S is of type t ∈ T
Dict_C_s = import_data(dir_data, "Dict_C_s"+str(S))                 # Computing resources required by SFCR s ∈ S
Dict_Lreq_s = import_data(dir_data, "Dict_Lreq_s"+str(S))           # E2E latency requirement of SFCR s ∈ S
Dict_Beta_s = import_data(dir_data, "Dict_Beta_s"+str(S))           # Bandwidth capacity required by SFCR s ∈ S
users_sourceBS = import_data(dir_data, "users_sourceBS")            # Access node (rs ∈ Nr) of SFCR s ∈ S

    
# Previous Placement data
Xtic = import_data(dir_data, "Xtic")                                # VNF deployed on node c
Zsfc = import_data(dir_data, "Zsfc")                                # SFC  mapped on node c

U = [x[:2] for x in Xtic]                                           # set of already deployed instances (i,t,c)
U = sorted(U , key=lambda k: [k[1], k[0]])

# Nodes
# Candidate locations
Nc = import_data(dir_data, "ListMECInstalled")                      # candidate coordenadas
Nc = sorted(Nc , key=lambda k: [k[1], k[0]])
Dict_C_c = import_data(dir_data, "Dict_C_c")                        # Processing capacity at candidate location c ∈ Nc


# access nodes
Nr = import_data(dir_data, "BS_pos")                                # access node coordinates 
Nr = sorted(Nr , key=lambda k: [k[1], k[0]])

 # Set of all nodes
N = Nc + Nr                                                        

#Importing paths data 
Edges = import_data(dir_data,"Edges")                               # dict: key=nodes_id (en & ran) and value latency
Dict_Links_Capacity = import_data(dir_data,"Dict_Links_Capacity")   # dictkey nodes_id (en & ran) and value bandwidth
Paths = import_data(dir_data,"Paths")
Paths_nm = import_data(dir_data,"Paths_nm")                         # P_n,m: dict with key equlas nodes_id (en & ran) and value number of paths
Paths_links_mapping = import_data(dir_data,'Paths_links_mapping')   # W_p_uv


# VNFs Data
Dict_C_t = import_data(dir_data,'Dict_C_t')             # Processing capacity of VNF of type t ∈ T         
Dict_D_proc = import_data(dir_data,'Dict_D_proc')       # d_t: Processing delay of VNF of type t ∈ T                            
M_t_imported = import_data(dir_data,'M_t')              # M_t: Maximum number of instances of type t ∈ T. Compute upon service demands and VNF capacity
M_t[4] = len(Nr)                                        # Updating number of BS 
T = params.Type_sfc                                     # T: Types of VNFs
T_extended = T+1                                        # Extending T dataset to include RANs as an special VNF type
T_dn = T_dn = params.T_dn                               # Processing time of data network nodes

# Preparing the Data

In [9]:
#Splitting the set of links between candidate nodes and ran-candidates
Edge_candidates = {}
Edge_rans_candidates = {}

for k,v in Edges.items():
    if k[0]<= len(Nc) and k[1]<= len(Nc):
        Edge_candidates[k]= v
    else:
        Edge_rans_candidates[k]=v

if len(Edge_candidates)+len(Edge_rans_candidates) != len(Edges):
    raise  
    
for k, v in Paths.items():
    Paths[k] = round(v, 2)

## Defining the general parameters for each type SFC

###  Types of PDU Sessions/ SFCR

+ Type 1:     
------| PSA |------
      
      
+ Type 2:     
------|I-UPF|-------| PSA |------
      
      
+ Type 3:     
------| M-UPF |-----|PSA |-----
               \̣____|PSA |-----
               
NOTE: All this types of VNFs has as initial VNF the RAN. In other words, the source nodes are considered as a type of VNFs. In the SFCR model the RANs are the last VNF and have t =4

### SFCR Parameters:
+ N_sfc_type: Number of CSFR types (i.e., 3)
+ B_sfc_type: Number of branches for each SFCR type
+ F_sfc_type: Set of VNFs for each SFCR type
+ T_sfc_type: Parameter indicating the type of VNF f in for each SFCR type
+ P_fbs_sfc_type: Parameter indicating if VNF f is present in branch b for each SFCR type
+ O_sbfg_sfc_type: Parameter indicating if VNF f goes before g in branch b for each SFCR type

### SFCR Requirements:
+ L_s: Latency requirement of service s
+ C_s: Capacity demand of servise s
+ Source_s: Source access node/AP of service s (List_ran_user (s,f,t,m,c) )
+ Bw_s: Bandwidth demand of service s

In [11]:
#  Parameters for type three of SFCR

if T == 3: # T: Number of VNFs types 1:PSA, 2:IUPF, 3: M-IUPF, 4:RAN
    Ran_type = 4
    B_sfc_type = [1,1,2]                       # No. of branches in each type of SFC
    F_sfc_type = [[1,2],[1,2,3],[1, 2, 3, 4]]  # Set of VNFs in each type of SFC
    T_sfc_type = [[1,4],[2,1,4],[3,1,1,4]]     # Type for each VNF in SFC 
    P_fbs_sfc_type = [[[1,1]], 
                      [[1,1,1]], 
                      [[1,1,0,1],[1,0,1,1]]]  # Indicates if a VNF is present or not in a branch: Index[type,branch,vnf]
    O_sbfg_sfc_type  = [[[[0,0], [1,0]]],
                        [[[0,1,0],[0,0,0],[1,0,0]]],
                        [[[0,1,0,0],[0,0,0,0],[0,0,0,0],[1,0,0,0]],
                        [[0,0,1,0],[0,0,0,0],[0,0,0,0],[1,0,0,0]]]]  #Indicates if a VNF f goes before g in a branch
    E_sfc_type = [[(2,1)],
                  [(3,1), (1,2)],
                  [(4,1), (1,2), (1,3)]]   #virtual links between VNF instances in set F

#### Defining PDU sessions  parameters

In [12]:
#Generating the data of each session PDU (demand, A-UPF number, BS source)
List_F_s = []
List_T_s = []
List_E_s = []
List_P_s = []
List_O_s = []
List_B_s = []

for k,v in Dict_Tsfc_s.items():
    List_F_s.append(F_sfc_type[v-1])
    List_T_s.append(T_sfc_type[v-1])
    List_E_s.append(E_sfc_type[v-1])
    List_P_s.append(P_fbs_sfc_type[v-1])
    List_O_s.append(O_sbfg_sfc_type[v-1])
    List_B_s.append(B_sfc_type[v-1])
    
Branches = sum(List_B_s)

# Pyomo model 

In [ ]:
# Creating the model 

model = ConcreteModel()

## Defining the Sets

### Basic Sets

In [14]:
#set of PDU sessions
model.del_component( 'S')
model.S = RangeSet(1, S)


#Set of candidates nodes
model.del_component( 'Nc')     
model.Nc = RangeSet(1, len(Nc))


#Set of ran nodes
model.del_component( 'Nr')     
model.Nr = RangeSet(len(Nc)+1, len(Nc)+len(Nr))


#Set of all the nodes: aggregation points+candidates nodes
model.del_component( 'N')     
model.N = RangeSet(1, len(N))


# Set of network edges. It includes the links between Nc+APs and Nc
model.del_component( 'E')
model.del_component( 'E_index')
model.E = Set(dimen=2, initialize=list(Edges.keys()), ordered= True)


# Set of network edges. It includes the links between Nc and Nc
model.del_component( 'E_NcxNc')
model.del_component( 'E_NcxNc_index')    
model.E_NcxNc = Set(dimen=2, initialize=list(Edge_candidates.keys()), ordered= True)


# Set of network edges. It includes the links between APs and Nc
model.del_component( 'E_NrxNc')
model.del_component( 'E_NrxNc_index')
model.E_NrxNc = Set(dimen=2, initialize=list(Edge_rans_candidates.keys()), ordered= True)


# Set with the combination of all the paths (n,m,h)
model.del_component( 'Paths')
model.del_component( 'Paths_index')
model.del_component( 'Paths_index_index_0')
model.Paths = Set(dimen=3, initialize=Paths.keys())


# Sets with the paths splitted by type of nodes
List_Nc = [i+1 for i in range(len(Nc))]
Paths_cxc = {}
Paths_rxc = {}
for k,v in Paths.items():
    if k[0] in List_Nc and k[1] in List_Nc:
        Paths_cxc[k] = v
    else:
        Paths_rxc[k] = v
        
model.del_component( 'Paths_cxc')
model.del_component( 'Paths_cxc_index')
model.del_component( 'Paths_cxc_index_index_0')
model.Paths_cxc = Set(dimen=3, initialize=Paths_cxc.keys()) 

model.del_component( 'Paths_rxc')
model.del_component( 'Paths_rxc_index')
model.del_component( 'Paths_rxc_index_index_0')
model.Paths_rxc = Set(dimen=3, initialize=Paths_rxc.keys()) 


# Set with the combination of all endpoints (n,m)
model.del_component( 'Paths_nm')
model.del_component( 'Paths_nm_index')
Dict_Path_nm = {}
for k,v in Paths_nm.items():
    Dict_Path_nm[k]= [i+1 for i in range(v)]
        
model.Paths_nm = Set(dimen=2, initialize=Dict_Path_nm.keys())


# Set with the combination of all endpoints (n,m), splitted by node type
Dict_Path_nm_cxc = {}
Dict_Path_nm_rxc = {}
for k in Paths.keys():
    if k in Paths_cxc.keys():
        Dict_Path_nm_cxc[k[:2]] = [i+1 for i in range(Paths_nm[k[:2]])]
    else:
        Dict_Path_nm_rxc[k[:2]] = [i+1 for i in range(Paths_nm[k[:2]])]

model.del_component( 'Paths_nm_cxc')
model.del_component( 'Paths_nm_cxc_index')        
model.Paths_nm_cxc = Set(dimen=2, initialize=Dict_Path_nm_cxc.keys())

model.del_component( 'Paths_nm_rxc')
model.del_component( 'Paths_nm_rxc_index')        
model.Paths_nm_rxc = Set(dimen=2, initialize=Dict_Path_nm_rxc.keys())


# Set with the number of paths bertween all endpoints (n,m)
model.del_component( 'Paths_nmh')
model.del_component( 'Paths_nmh_index')    
model.Paths_nmh = Set(model.Paths_nm, initialize=Dict_Path_nm)


#Set of VNFs types (UPFs)
model.del_component( 'T')
model.T = RangeSet(1, T)


# Set of max. number of available VNF instances for each type of VNF
model.del_component( 'I_t')
Dict_M_t = {}
for k,v in M_t.items():
    if k!=4:
        Dict_M_t[k] = [i+1 for i in range(v)]
model.I_t = Set(model.T, initialize= Dict_M_t) 


# Set of deployd VNF instances, (t,i) in U
model.del_component( 'U')
model.del_component( 'U_index')
model.U = Set(dimen=2, initialize= U)


#Sets of VNFs (basic and extended) forming a SFCR
model.del_component( 'F')
Dict_VNF_s = {}
Dict_VNF_s_extended = {}
for s in model.S:
    Dict_VNF_s[s] = List_F_s[s-1][:-1]
    Dict_VNF_s_extended[s] = List_F_s[s-1]

model.F = Set(model.S, initialize= Dict_VNF_s)


# Set of branches in a SFC s
model.del_component( 'B_s')
Dict_Branches={}
for s in model.S:
    Dict_Branches[s] = [b+1 for b in range(List_B_s[s-1])]
model.B_s = Set(model.S, initialize= Dict_Branches)

### Combined Sets

In [15]:
# set TM 
model.del_component( 'TI')
model.del_component( 'TI_index')
List_TM = [(k,v1) for k,v in  Dict_M_t.items() for v1 in v if k!=4]  # excluyendo RANs
model.TI = Set(dimen=2, initialize= List_TM)


#Sets of VNFs (basic and extended) forming a SFCR
Dict_VNF_s = {}
Dict_VNF_s_extended = {}
for s in model.S:
    Dict_VNF_s[s] = List_F_s[s-1][:-1]
    Dict_VNF_s_extended[s] = List_F_s[s-1]

model.del_component( 'F')
model.F = Set(model.S, initialize= Dict_VNF_s)


# SEt of SFCR and VNFs combined (s,f)
model.del_component( 'SF')
model.del_component( 'SF_index')
SF = [(s, i) for s in model.S for i in List_F_s[s-1][:-1]]  # List of tuples (user_id, vnf_id)
model.SF = Set(initialize= SF, ordered=True)


# FGS is a list with [s,f,g] combination between any two VNFs f and g of each SFCR s
model.del_component( 'SFG')
model.del_component( 'SFG_index')
model.del_component( 'SFG_index_index_0')
SFG = [(s,f,g) for s in model.S for f in model.F[s] for g in model.F[s]]
model.SFG = Set(dimen=3, initialize= SFG)


#Set with the combinations between sfcr and branches (s,b)
SB = [(s,b) for s in model.S for b in model.B_s[s] ]
model.del_component( 'SB')
model.del_component( 'SB_index')
model.SB = Set(dimen=2, initialize=SB)


# Set with the combinations between SFC s, Branches b and VNFs f (s,b,f)
model.del_component( 'SBF')
model.del_component( 'SBF_index')
model.del_component( 'SBF_index_index_0')
SBF = [(s,b,f) for s in model.S for b in model.B_s[s] for f in model.F[s]]
model.SBF = Set (dimen=3, initialize=SBF)

## Defining the Parameters

### Network Parameters

In [16]:
# VNF t capacity    
model.del_component( 'C_t')
C_t = {}
for t in model.T:
    C_t[t] = Dict_C_t[t]
model.C_t = Param(model.T, initialize=C_t)


#MaxNumber of VNF instances for each type (M_t)
model.del_component( 'MaxM_t')
M_t_upf = {}
for k,v in M_t.items():
    if k!=4:
        M_t_upf[k] = v
model.MaxM_t = Param(model.T, initialize= M_t_upf)


# Server capacity
model.del_component( 'C_c') 
model.C_c = Param(model.Nc, initialize=Dict_C_c)


# Propagation Latency in a path. Includes all the links between APs and Nc (N)
model.del_component( 'D_prop')
model.del_component( 'D_prop_index')
model.D_prop = Param(model.Paths, initialize=Paths)


#Processing Latency for each VNF type
model.del_component( 'D_proc')
D_p = {}
for t in model.T:
    D_p[t] = Dict_Tproc[t]
model.D_proc = Param(model.T, initialize= D_p)


# Link Bandwidth
model.del_component( 'Link_bw')
model.del_component( 'Link_bw_index')
model.Link_bw = Param(model.E, initialize=Dict_Links_Capacity)


# Path_link mapping
model.del_component( 'W_nmh_uv')
model.del_component( 'W_nmh_uv_index')
model.del_component( 'W_nmh_uv_index_index_0')
model.del_component( 'W_nmh_uv_index_index_0_index_0')
model.del_component( 'W_nmh_uv_index_index_0_index_0_index_0')

Paths_links_mapping_dict = {}
for k,v in Paths_links_mapping.items():
    for k1,v1 in v.items():
        Paths_links_mapping_dict[(k[0],k[1], k[2], k1[0], k1[1])]=v1

model.W_nmh_uv = Param(model.Paths, model.E, initialize=Paths_links_mapping_dict)

### Previous placement parameters

In [17]:
# VNf instances :  Xtic[t,i,c]=1 if instance i of VNF type t was previously deployed at node c
model.del_component( 'Xtic')
model.del_component( 'Xtic_index')
model.del_component( 'Xtic_index_index_0')
Dict_Xtic = {}
for t, i in model.TI:
    for c in model.Nc:
        if (t,i,c) in Xtic:
            Dict_Xtic[(t,i,c)] = 1
        else:
            Dict_Xtic[(t,i,c)] = 0
            
model.Xtic = Param(model.TI, model.Nc, initialize=Dict_Xtic)


model.del_component( 'Zsfc')
model.del_component( 'Zsfc_index')
model.del_component( 'Zsfc_index_index_0')
Dict_Zsfc = {}
for s, f in model.SF:
    for c in model.Nc:
        if (s,f,c) in Zsfc:
            Dict_Zsfc[(s,f,c)] = 1
        else:
            Dict_Zsfc[(s,f,c)] = 0
            
model.Zsfc = Param(model.SF, model.Nc, initialize=Dict_Zsfc)

### PDU Session Parameters

In [18]:
# SFCR processing demand 
model.del_component( 'C_s')
model.C_s = Param(model.S, initialize=Dict_C_s)


# SFCR latency demand 
model.del_component( 'Lreq_s')
model.Lreq_s = Param(model.S, initialize=Dict_Lreq_s)


#Link_Bandwidth demand
model.del_component( 'beta_s')
model.beta_s = Param(model.S, initialize=Dict_Beta_s)


# VNF presence in a branch P[s,b,f]= 1 if vnf f is present in branch b of sfc s
model.del_component( 'P_sbf')
model.del_component( 'P_sbf_index')
model.del_component( 'P_sbf_index_index_0')

P_s= {}
for s,b,f in model.SBF: 
    P_s[(s,b,f)] = List_P_s[s-1][b-1][f-1]    
model.P_sbf = Param(model.SBF, initialize=P_s)


#T_stf= 1 if VNF f in SFCs s is of type t
model.del_component( 'T_sft')
model.del_component( 'T_sft_index')
model.del_component( 'T_sft_index_index_0')  

Dict_T_sft ={} #key=(s,f,t), value= 1 if f is of type t, 0 otherwise
for s,f in model.SF:
    for t in model.T:
        if t == List_T_s[s-1][f-1]:
            Dict_T_sft[(s,f,t)]= 1
        else:
            Dict_T_sft[(s,f,t)]= 0             
model.T_sft = Param(model.SF, model.T, initialize=Dict_T_sft)


# Node Supported Type of VNF
Dict_S_nt = {}  #dict with keys (n,t) and binary values, been 1 if node n support VNF type t, 0 otherwise
for n in model.Nc:
    for t in model.T:
        Dict_S_nt[(n,t)]=1
            
model.del_component( 'NodeSupportType')
model.del_component( 'NodeSupportType_index')
model.NodeSupportType = Param(model.N, model.T, initialize=Dict_S_nt)


#O_sbfg= 1 if vnf f goes before g in branch b of sfc s
model.del_component( 'SBFG')
model.del_component( 'SBFG_index')
model.del_component( 'SBFG_index_index_0')    
model.del_component( 'SBFG_index_index_0_index_0')

Dict_O_sbfg = {}
for s,b in model.SB:
    for f in model.F[s]:        
        for g in model.F[s]:
            Dict_O_sbfg[(s,b,f,g)]= List_O_s[s-1][b-1][f-1][g-1]
model.SBFG = Set(dimen=4, initialize=Dict_O_sbfg.keys())


model.del_component( 'O_sbfg')
model.del_component( 'O_sbfg_index')
model.del_component( 'O_sbfg_index_index_0')    
model.del_component( 'O_sbfg_index_index_0_index_0')
model.O_sbfg = Param(model.SBFG,  initialize=Dict_O_sbfg)


#Ô_sbg= 1 if vnf g is the first vnf in branch b of sfc s
model.del_component( 'Ô_sbg')
model.del_component( 'Ô_sbg_index')
model.del_component( 'Ô_sbg_index_index_0') 
Dict_O_sbg = {}
for s,b in model.SB:
    for g in model.F[s]: 
        Dict_O_sbg[(s,b,g)] = List_O_s[s-1][b-1][len(model.F[s])][g-1]  #len(model.F[s])= position of ran in List_O_s

model.Ô_sbg = Param(model.SBF,  initialize=Dict_O_sbg)


# Paths_source_s Has a list with all the paths that have the bs of s as source.
model.del_component( 'Paths_source_s')
model.del_component( 'Paths_source_s_index')
model.del_component( 'Paths_source_s_index_index_0')
Dict_s_sourcepaths = {}
for s in model.S:
    source = Nr.index(users_sourceBS[s])+len(Nc)+1
    for p in Paths_rxc.keys():
        if source == p[0]:
            Dict_s_sourcepaths.setdefault(s, []).append(p)
            
model.Paths_source_s = Set(model.S, initialize= Dict_s_sourcepaths)

## Defining the Variables

In [19]:
# w[c]= 1 if node c is open
model.del_component( 'w') 
model.w = Var(model.Nc,  within=Binary, initialize=0)


# x[m,t,c]=1 if instance m of VNF type t is deployed at node c
model.del_component( 'x')
model.del_component( 'x_index')
model.del_component( 'x_index_index_0')
model.x = Var(model.TI, model.Nc, within=Binary, initialize=0)


# a[s,f,t,i,c]=1 if VNF f of SFCR s is assigned to instance m of VNF type t is deployed at node c
model.del_component( 'a')
model.del_component( 'a_index')
model.del_component( 'a_index_index_0')
model.del_component( 'a_index_index_0_index_0')
model.del_component( 'a_index_index_0_index_0_index_0')
model.a = Var(model.SF, model.TI, model.Nc, within=Binary, initialize=0)


# z[s,f,c] = 1 if the VNF f of theSFC s is assigned to node c
model.del_component( 'z')
model.del_component( 'z_index')
model.del_component( 'z_index_index_0')
model.z = Var(model.SF, model.Nc, within=Binary, initialize=0)


# y[s,b,f,g,n,m,h]= 1 if path h between nodes (i,j) is used to route traffic between VNFs f and g in branch b 
                    # of SFCR s 
model.del_component( 'y')
model.del_component( 'y_index')
model.del_component( 'y_index_index_0')
model.del_component( 'y_index_index_0_index_0')
model.del_component( 'y_index_index_0_index_0_index_0')
model.del_component( 'y_index_index_0_index_0_index_0_index_0')
model.y = Var(model.SFG, model.Paths_cxc, within=Binary, initialize=0)


# ŷ[s,b,g,source,m,h]= 1 if path h between nodes (source,j) is used to route traffic between the BS and VNF g 
                        # in branch b of SFCR s 
model.del_component( 'ŷ')
model.del_component( 'ŷ_index')
model.del_component( 'ŷ_index_index_0')
model.del_component( 'ŷ_index_index_0_index_0')
model.del_component( 'ŷ_index_index_0_index_0_index_0')
model.ŷ = Var(model.SF, model.Paths_rxc, within=Binary, initialize=0)

### Linearization variables

In [20]:
# delta[s,b,f,g,i,j]=1 if virtual link (i,j) may route traffic between VNFs f and g of SFCR s. To linearize.
model.del_component( 'delta')
model.del_component( 'delta_index')
model.del_component( 'delta_index_index_0')
model.del_component( 'delta_index_index_0_index_0')
model.del_component( 'delta_index_index_0_index_0_index_0')
model.delta = Var( model.SFG, model.Paths_nm,  within=Binary, initialize=0)


# model.delta_n[t] = 1 if there is at least a new instance of type t deployed. Otherwise, it's zero.
model.del_component( 'delta_n')
model.delta_n = Var(model.T, within=Binary, initialize= 0)


# model.m[t,i,c,e] = 1 if vnf t,i was migrated from c fo e
model.del_component( 'm')
model.del_component( 'm_index')
model.del_component( 'm_index_index_0')
model.del_component( 'm_index_index_0_index_0')
model.m = Var(model.U, model.Nc, model.Nc, within= Binary, initialize=0)


# n[t,i]= 1 if instnace i of type t is new
model.del_component( 'n') 
model.del_component( 'n_index') 
model.n = Var(model.TI,  within=Binary, initialize=0)


#r[s] = 1 if PDU session s has been reassigned. At least one vnf has change of location
model.del_component( 'r')
model.r = Var(model.S, within=Binary, initialize=0)

## Defining the Objective

Initially the objective is minimize the number of VNF instances

- Node activation cost
$C_{act} = \sum \limits_{n\in N_c} \Psi_{a}^{n} \cdot w_{n}$

-  VNF deployment cost 
$  C_{dep} = \sum \limits_{i \in I_t}\sum \limits_{t\in T} \Psi_{d}^{t} \cdot v_{i,t}$

- VNF running cost
$ C_{run} = \sum \limits_{i \in I_t}\sum \limits_{t\in T} \sum \limits_{n \in N_c} \Psi_r^{t,n} \cdot x_{i,t,n}$

- VNF migration cost
$ C_{mig}= \sum \limits_{(i,t) \in F} \sum \limits_{  n^{\tiny \prime} \in N_c}\sum \limits_{  n \in N_c} \Psi_m^{t, n^{\tiny\prime},n} \cdot m_{n^{\tiny \prime},n}^{i,t}$

- Routing cost
$ C_{rou} =  \sum\limits_{f,g \in F_s^+}  \sum\limits_{ s \in S} \sum\limits_{ p \in P}  \Psi^{s,p} \cdot d_{p} \cdot  y_{p}^{f,g,s}$

- Session reassignment cost
$C_{rea} =  \sum\limits_{  s \in S} \Psi ^s \cdot r_s$

In [22]:
model.del_component( 'Objective')

# weight factors
w_a = params.WEIGHT_D_EN            # NODE ACTIVATION
w_r = params.WEIGHT_D_RUN           # VNF operation
w_d = params.WEIGHT_D_DEP           # VNF deployment
w_m = params.WEIGHT_D_MIG           # VNF migration
w_l = params.WEIGHT_D_ROU           # traffic routing
w_s = params.WEIGHT_D_REA           # session reassignment

def Objective_rule(model):
    
    # Node activation cost
    Activation = sum( model.w[c] for c in model.Nc)
    Nor_Activation = Activation/len(Nc)   # maximum number of servers
    
    # VNF Running cost
    Running = sum(model.x[t,i,c] for t,i in model.TI for c in model.Nc)
    Max_Num_Instances = sum(M_t.values())-len(Nr) 
    Nor_Running = Running/Max_Num_Instances
    
    # VNF deploymnet cost:new deployments
    Deployment = sum(model.n[t,i] for t,i in model.TI )
    Max_num_newinstances = Max_Num_Instances - len(Xtic)
    Nor_Deployment = Deployment/Max_num_newinstances
    
    # Migration cost
    Migration = sum(model.m[t,i,c,e] for c in model.Nc for e in model.Nc if c!=e 
                    for t,i in model.U )
    Max_num_migrations = len(Xtic)
    Nor_Migration = Migration/Max_num_migrations

    # Routing Cost
    Latency_bs_nf1 = sum(model.D_prop[n,m,h]*model.ŷ[s,g,n,m,h]*model.Ô_sbg[s,b,g]              # latency in the segment RAN-VNF1
                             for s,b in model.SB  for g in model.F[s]  
                             for n,m,h in model.Paths_rxc)
    Latency_nfs = sum(model.D_prop[n,m,h]*model.y[s,f,g,n,m,h]*model.O_sbfg[s,b,f,g]            # Latency between VNFs
                                          for s,b in model.SB for f in model.F[s] for g in model.F[s]
                                          for n,m,h in model.Paths_cxc) 
    Latency_oneway = Latency_bs_nf1 + Latency_nfs                                
    Nor_Latency_oneway = Latency_oneway/(100*sum(List_B_s))

    #Reassigned sessions
    Reassignments = sum(model.r[s] for s in model.S)
    Nor_Reassignments = Reassignments/S
    
    
    obj = (w_a*Nor_Activation + w_r*Nor_Running + w_d*Nor_Deployment + w_m*Nor_Migration +
           w_l*Nor_Latency_oneway + w_s*Nor_Reassignments)
    
    return obj

model.Objective = Objective(rule=Objective_rule, sense=minimize)

## Defining the Constraints

##### C1: Node Capacity

$\sum \limits_{t\in T}\sum \limits_{m\in I_t}C_t*x_{t,i,c}<= C_c   \hspace{30pt} \forall c \in N_c$

In [23]:
model.del_component( 'NodeCapacity')

def NodeCapacity_rule(model,c):
    return sum(model.C_t[t]*(model.x[t,i,c]) for t,i in model.TI) <= model.C_c[c]

model.NodeCapacity = Constraint(model.Nc, rule=NodeCapacity_rule)

##### C2: VNF Capacity

$ \sum \limits_{s\in S}\sum \limits_{f \in F_s} C_s \cdot a_{t,i,n}^{s,f} \leq C_t  \hspace{30pt}\forall i \in I_t, \forall t \in T,  \forall n \in N_c $

In [25]:
model.del_component( 'VNFCapacity')
model.del_component( 'VNFCapacity_index')
model.del_component( 'VNFCapacity_index_index_0')

def VNFCapacity_rule(model,t,i,c):
    return sum((model.C_s[s]*model.a[s,f,t,i,c]) for s,f in model.SF) <= alpha*model.C_t[t]

model.VNFCapacity = Constraint(model.TI, model.Nc, rule=VNFCapacity_rule)

##### C3: Link Capacity

$ \sum\limits_{s\in S} \sum\limits_{f,g\in F_s^+}\sum\limits_{p\in P} \beta_s \cdot y_{p}^{s,f,g} \cdot W_{u,v}^{p}\leq \beta_{u,v} \hspace{30pt} \forall (u,v) \in E $

In [27]:
model.del_component( 'LinkCapacity')
model.del_component( 'LinkCapacity_index')

def LinkCapacity_rule(model, i,j):
   
    link_cap = (sum(model.beta_s[s]*model.y[s,f,g,n,m,h]* model.W_nmh_uv[n,m,h,i,j] 
                    for s,f,g in model.SFG for n,m,h in model.Paths_cxc)+sum(model.beta_s[s]*model.ŷ[s,g,n,m,h]* model.W_nmh_uv[n,m,h,i,j] 
                                                                        for s,g in model.SF for n,m,h in model.Paths_rxc))
                    
    return link_cap <= model.Link_bw[i,j]

model.LinkCapacity = Constraint(model.E, rule=LinkCapacity_rule)

##### C4: Maximum number of intances of a type t

$ \sum \limits_{i \in I_t} \sum \limits_{n \in N_c} x_{t,i,n} \leq I_t \hspace{30pt}  \forall t\in T$ 

In [29]:
model.del_component( 'MaxNUmVNFType')

def MaxNUmVNFType_rule(model,t):
    return sum(model.x[t,m,c] for m in model.I_t[t] for c in model.Nc)  <= model.MaxM_t[t]

model.MaxNUmVNFType = Constraint(model.T, rule=MaxNUmVNFType_rule)

##### C5: Mapping of a VNF Instance 

Each VNf instance must be mapped to only one node (server)

$\sum \limits_{c\in Nc}x_{t,i,c} \leq 1   \hspace{30pt} \forall t \in T, \forall i \in It$

In [31]:
model.del_component( 'MappingVNFInstance')
model.del_component( 'MappingVNFInstance_index')

def MappingVNFInstance_rule(model,t, i):
    return sum(model.x[t,i,c] for c in model.Nc) <= 1

model.MappingVNFInstance = Constraint(model.TI, rule=MappingVNFInstance_rule)

#### C: Node open

An instance of a VNF type can be deployed on a node only if this is open

$x_{t,i,c} \leq w_{c}   \hspace{30pt} \forall t \in T, \forall i \in I_t, \forall c \in N$

In [32]:
model.del_component( 'Opening')
model.del_component( 'Opening_index')
model.del_component( 'Opening_index_index_0')

def Opening_rule(model,c,t,i):
    return model.x[t,i,c] <= model.w[c]*model.NodeSupportType[c,t] 

model.Opening = Constraint(model.Nc, model.TI, rule=Opening_rule)

#### C: Node open part2

A node is open if it has deployed at east one vnf instance

$ \sum \limits_{\forall t \in T} \sum \limits_{\forall i \in I_t}  x_{t,i,c} \geq w_{c}   \hspace{30pt}   \forall c \in N$

In [34]:
model.del_component( 'Opening2')

def Opening2_rule(model,c):
    return sum(model.x[t,i,c] for t,i in model.TI) >= model.w[c]

model.Opening2 = Constraint(model.Nc,  rule=Opening2_rule)

#### C: maximum number of new instances deployed during a reconfiguration

$ \sum \limits_{i \in I_t} v_{t,i} = [\sum \limits_{n \in N_c}\sum\limits_{i \in I_t} x_{t,i,n} - \sum \limits_{n \in N_c}\sum\limits_{i \in I_t} X_{t,i,n}]^+  \hspace{30pt} \forall t \in T$


In [35]:
# linear form

model.del_component( 'Num_newvnfs1')

def Num_newvnfs1_rule(model,t):
    Num_newinstances = sum(model.x[t,i,c] for i in model.I_t[t] for c in model.Nc) - sum(model.Xtic[t,i,c] for i in model.I_t[t] 
                                                                                       for c in model.Nc)
    return model.delta_n[t] >= Num_newinstances/M_t[t]

model.Num_newvnfs1 = Constraint(model.T, rule=Num_newvnfs1_rule)   



model.del_component( 'Num_newvnfs2')

def Num_newvnfs2_rule(model,t):
    Num_newinstances = sum(model.x[t,i,c] for i in model.I_t[t] for c in model.Nc) - sum(model.Xtic[t,i,c] 
                                                                                       for i in model.I_t[t] 
                                                                                       for c in model.Nc)
    return model.delta_n[t] <= (1 - (1-Num_newinstances)/(M_t[t] + 1))
        
model.Num_newvnfs2 = Constraint(model.T, rule=Num_newvnfs2_rule)   



model.del_component( 'Num_newvnfs')

def Num_newvnfs_rule(model,t):
    Num_newinstances = sum(model.x[t,i,c] for i in model.I_t[t] for c in model.Nc) - sum(model.Xtic[t,i,c] 
                                                                                       for i in model.I_t[t] 
                                                                                       for c in model.Nc)
    return sum(model.n[t,i] for i in model.I_t[t]) == model.delta_n[t]*Num_newinstances

model.Num_newvnfs = Constraint(model.T, rule=Num_newvnfs_rule)  

#### C: New instance of type t deployed

$ v_{t,i} = (\sum \limits_{n \in N_c} x_{t,i,n} - \sum \limits_{n \in N_c} X_{t,i,n})^+ \hspace{30 pt} \forall i \in I_t, \forall t \in T$


In [42]:
# linear form
 
model.del_component( 'NewInstance1')
model.del_component( 'NewInstance1_index')

def NewInstance1_rule(model, t,i):
    newinstance = sum(model.x[t,i,c] for c in model.Nc) - sum(model.Xtic[t,i,c] for c in model.Nc)
    
    return model.n[t,i]  >=  newinstance
    
model.NewInstance1 = Constraint( model.TI, rule= NewInstance1_rule)



model.del_component( 'NewInstance2')
model.del_component( 'NewInstance2_index')

def NewInstance2_rule(model, t,i):
    newinstance = sum(model.x[t,i,c] for c in model.Nc) - sum(model.Xtic[t,i,c] for c in model.Nc)
    
    return model.n[t,i] <= (1 + newinstance)/2 
    
model.NewInstance2 = Constraint( model.TI, rule= NewInstance2_rule)

#### C: migration

 $ m_{t,i,n^{\tiny\prime},n} =  x_{t,i,n}\cdot X_{t,i,n^{\tiny\prime}} \hspace{30 pt} \forall i \in I_t, \forall t \in T,  \forall n,n^{\tiny\prime} \in  N_c; n \neq n^{\tiny\prime}$

In [46]:
#Variant1:

model.del_component( 'Migration')
model.del_component( 'Migration_index')
model.del_component( 'Migration_index_index_0')
model.del_component( 'Migration_index_index_0_index_0')

def Migration_rule(model,t,i,c,e):
    if c!=e:
        return model.m[t,i,c,e] == model.x[t,i,e]*model.Xtic[t,i,c]
    else:
        return model.m[t,i,c,e] <= 0

model.Migration = Constraint(model.U, model.Nc, model.Nc, rule=Migration_rule)

#### C: Reassignment of session

$ r_s = 1 \Leftrightarrow |F_s| -   \sum \limits_{f \in F_s}\sum \limits_{n \in N_c} a_n^{f,s} \cdot \bar A_n^{f,s} \geq 1 \hspace{30 pt}  \forall s \in S $

In [48]:
#linear form

model.del_component( 'Reassignment1')

def Reassignment1_rule(model, s):
    return ( sum(model.z[s,f,c]*model.Zsfc[s,f,c] for f in model.F[s] for c in model.Nc) >= 
            len(model.F[s])*(1-model.r[s]) )

model.Reassignment1 = Constraint(model.S, rule= Reassignment1_rule)



model.del_component( 'Reassignment2')

def Reassignment2_rule(model, s):
    return (sum(model.z[s,f,c]*model.Zsfc[s,f,c] for f in model.F[s] for c in model.Nc) <= 
            len(model.F[s])-model.r[s])
         
model.Reassignment2 = Constraint(model.S, rule= Reassignment2_rule)

##### C: Mapping of  VNF service Requests in a  SFCR

Each VNf service requested by a SFCR must be mapped to exactly one VNF instance 

$\sum \limits_{t\in T}\sum \limits_{i\in It}\sum \limits_{c\in Nc}a_{t,i,c}^{s,b} = 1   \hspace{30pt} \forall f \in Fs, \forall s \in S$

In [52]:
model.del_component( 'MappingVNFRequest')
model.del_component( 'MappingVNFRequest_Request')

def MappingVNFRequest_rule(model,s, f):
    return sum(model.a[s,f,t,m,c] for t,m in model.TI for c in model.Nc) == 1

model.MappingVNFRequest = Constraint(model.SF, rule=MappingVNFRequest_rule)

##### C: No empty VNF instances
A VNF service reuested by a SFC can be assigned to an instance of a VNF at a server if this is already deployed and is of the requested type

$  x_{t,i,c}\leq  \sum \limits_{\forall s \in S} \sum \limits_{\forall f \in F_s} a_{t,i,c}^{s,f}  \hspace{30pt}  \forall t\in T, \forall i\in It, \forall c\in Nc$

In [54]:
model.del_component( 'NoemptyVNF')
model.del_component( 'NoemptyVNF_index')
model.del_component( 'NoemptyVNF_index_index_0')

def Noempty_rule(model, t,i,c):
    return model.x[t,i,c] <= sum(model.a[s,f,t,i,c] for s,f in model.SF)

model.NoemptyVNF = Constraint(model.TI, model.Nc, rule = Noempty_rule)

##### C: VNF f in SFCR s must be Assigned to a instance of its same type

A VNF service reuested by a SFC can be assigned to an instance of a VNF at a server if this is already deployed and is of the requested type

$ a_{t,i,c}^{s,b} \leq x_{t,i,c} \cdot T_s^{f,t}   \hspace{30pt} \forall f \in F_s, \forall s \in S, \forall t\in T, \forall i\in It, \forall c\in Nc$

In [56]:
model.del_component( 'VNFAssignment')
model.del_component( 'VNFAssignment_index')
model.del_component( 'VNFAssignment_index_index_0')
model.del_component( 'VNFAssignment_index_index_0_index_0')
model.del_component( 'VNFAssignment_index_index_0_index_0_index_0')
 
def VNFAssignment_rule(model,s,f,t,i,c):
    return model.a[s,f,t,i,c] <= model.x[t,i,c]*model.T_sft[s,f,t]

model.VNFAssignment = Constraint(model.SF, model.TI, model.Nc, rule=VNFAssignment_rule)

##### C: VNF assigned to an instance in its assigned node

A VNF service reuested by a SFC can be assigned to an instance of a VNF at a server if this has been mapped to the server

$ a_{t,i,c}^{s,b} \leq z_{s,f,c}   \hspace{30pt} \forall f \in F_s, \forall s \in S, \forall t\in T, \forall i\in It, \forall c\in Nc$

In [58]:
model.del_component( 'VNF_Branch')
model.del_component( 'VNF_Branch_index')
model.del_component( 'VNF_Branch_index_index_0')
model.del_component( 'VNF_Branch_index_index_0_index_0')
model.del_component( 'VNF_Branch_index_index_0_index_0_index_0')

def VNF_Branch_rule(model, s,f,t,i,c):
    return model.a[s,f,t,i,c] <= model.z[s,f,c] 

model.VNF_Branch = Constraint(model.SF, model.TI, model.Nc, rule= VNF_Branch_rule)

In [59]:
# model.VNF_Branch.display()

##### C: VNF assigned to an instance in its assigned node

A VNF service reuested by a SFC can be assigned to an instance of a VNF at a server if this has been mapped to the server

$ z_{s,f,c} \leq \sum \limits_{\forall t\in T}\sum \limits_{ \forall i\in It} a_{t,i,c}^{s,b}   \hspace{30pt} \forall f \in F_s, \forall s \in S, \forall c\in Nc$

In [60]:
model.del_component( 'A_Z')
model.del_component( 'A_Z_index')
model.del_component( 'A_Z_index_index_0')

def A_Z_rule(model, s,f,c):
    return  model.z[s,f,c] <= sum(model.a[s,f,t,i,c] for t,i in model.TI)

model.A_Z = Constraint(model.SF, model.Nc, rule= A_Z_rule)

In [61]:
# model.A_Z .display()


#### C : Link_Order

Guarantees the existence of a path in the required direction between two consecutive VNFs in a sfc

$ \sum\limits_{(i,j) \in E} y_{i,j}^{f,g,b,s} \geq O_{s,b,f,g} \hspace{30pt} \forall b \in B_s,\forall s \in S, \forall f,g \in F_s$

In [62]:
model.del_component( 'LinkOrder1')
model.del_component( 'LinkOrder1_index')
model.del_component( 'LinkOrder1_index_index_0')
model.del_component( 'LinkOrder1_index_index_0_index_0')

def LinkOrder1_rule(model, s,b,f,g):
    return model.O_sbfg[s,b,f,g] <= sum(model.y[s,f,g,n,m,h] for n,m,h in model.Paths_cxc)

model.LinkOrder1 = Constraint(model.SBFG, rule= LinkOrder1_rule)


#### C : Link_Order_2


$ \sum\limits_{(i,j) \in E} y_{i,j}^{f,g,b,s} \leq 1 \hspace{30pt} \forall b \in B_s,\forall s \in S, \forall f,g \in F_s$

In [63]:
model.del_component( 'LinkOrder2')
model.del_component( 'LinkOrder2_index')
model.del_component( 'LinkOrder2_index_index_0')
model.del_component( 'LinkOrder2_index_index_0_index_0')

def LinkOrder2_rule(model, s,f,g):
    return 1 >= sum(model.y[s,f,g,n,m,h] for n,m,h in model.Paths_cxc)

model.LinkOrder2 = Constraint(model.SFG, rule= LinkOrder2_rule)


#### C : Special_Link_Order

Guarantees the existence of a path in the required direction between the source node (BS) and the first VNF in each branch of a sfc

$ \sum\limits_{(i,j) \in E} ŷ_{source,j}^{g,b,s} \geq Ô_{s,b,g} \hspace{30pt} \forall b \in B_s,\forall s \in S, \forall g \in F_s$

In [64]:
model.del_component( 'Special_LinkOrder1')
model.del_component( 'Special_LinkOrder1_index')
model.del_component( 'Special_LinkOrder1_index_index_0')
model.del_component( 'Special_LinkOrder1_index_index_0_index_0')

def Special_LinkOrder1_rule(model, s,b,g):
    return model.Ô_sbg [s,b,g] <= sum(model.ŷ[s,g,n,m,h] for n,m,h in model.Paths_source_s[s])

model.Special_LinkOrder1 = Constraint(model.SBF, rule= Special_LinkOrder1_rule)


#### C : Special_Link_Order2

Guarantees the existence of a path in the required direction between the source node (BS) and the first VNF in each branch of a sfc

$ \sum\limits_{(i,j) \in E} ŷ_{source,j}^{g,b,s} \geq Ô_{s,b,g} \hspace{30pt} \forall b \in B_s,\forall s \in S, \forall g \in F_s$

In [65]:
model.del_component( 'Special_LinkOrder2')
model.del_component( 'Special_LinkOrder2_index')
model.del_component( 'Special_LinkOrder2_index_index_0')
model.del_component( 'Special_LinkOrder2_index_index_0_index_0')

def Special_LinkOrder2_rule(model, s,g):
    return 1 >= sum(model.ŷ[s,g,n,m,h] for n,m,h in model.Paths_source_s[s])
#
model.Special_LinkOrder2= Constraint(model.SF, rule= Special_LinkOrder2_rule)

#### C: VNF_link

$ \sum\limits_{(i,j) \in E} y_{i,j}^{f,g,b,s} \leq (z_{s,f,i} \cdot z_{s,g,j})   \hspace{30 pt}
\forall b \in B_s,\forall s \in S, \forall f,g \in F_s,  \forall i \in N_c$

NOTE: I had to define the constraint using Nc instead of the Paths_nm because since Paths_nm includes the Rans and thereby the constraint gave an error because the set z was defined with Nc and it does not include all the nodes, that is, it does not include the RANs

In [66]:
model.del_component( 'VNF_link1')
model.del_component( 'VNF_link1_index')
model.del_component( 'VNF_link1_index_index_0')
model.del_component( 'VNF_link1_index_index_0_index_0')
model.del_component( 'VNF_link1_index_index_0_index_0_index_0')
model.del_component( 'VNF_link1_index_index_0_index_0_index_0_index_0')

def VNF_link1_rule(model, s,f,g,n,m):
    return model.z[s,f,n] >= model.delta[s,f,g,n,m]

model.VNF_link1 = Constraint(model.SFG, model.Nc, model.Nc, rule= VNF_link1_rule)


model.del_component( 'VNF_link2')
model.del_component( 'VNF_link2_index')
model.del_component( 'VNF_link2_index_index_0')
model.del_component( 'VNF_link2_index_index_0_index_0')
model.del_component( 'VNF_link2_index_index_0_index_0_index_0')
model.del_component( 'VNF_link2_index_index_0_index_0_index_0_index_0')

def VNF_link2_rule(model, s,f,g,n,m):
    return model.z[s,g,m]>=model.delta[s,f,g,n,m]
   
model.VNF_link2 = Constraint(model.SFG,  model.Nc, model.Nc, rule= VNF_link2_rule)


model.del_component( 'VNF_link3')
model.del_component( 'VNF_link3_index')
model.del_component( 'VNF_link3_index_index_0')
model.del_component( 'VNF_link3_index_index_0_index_0')
model.del_component( 'VNF_link3_index_index_0_index_0_index_0')
model.del_component( 'VNF_link3_index_index_0_index_0_index_0_index_0')

def VNF_link3_rule(model, s,f,g,n,m):
    if f!=g:
        return model.delta[s,f,g,n,m] >= (model.z[s,f,n] + model.z[s,g,m]-1)
                                             
    else:
        return model.delta[s,f,g,n,m] <= 0
    
model.VNF_link3 = Constraint(model.SFG, model.Nc, model.Nc,  rule= VNF_link3_rule)


model.del_component( 'VNF_link')
model.del_component( 'VNF_link_index')
model.del_component( 'VNF_link_index_index_0')
model.del_component( 'VNF_link_index_index_0_index_0')
model.del_component( 'VNF_link_index_index_0_index_0_index_0')
model.del_component( 'VNF_link_index_index_0_index_0_index_0_index_0')

def VNF_link_rule(model, s,f,g,n,m):
    return model.delta[s,f,g,n,m] >= sum(model.y[s,f,g,n,m,h] for h in model.Paths_nmh[n,m])

model.VNF_link = Constraint(model.SFG, model.Paths_nm_cxc, rule= VNF_link_rule)

#### C: Special_VNF_link

Adapts previous constraint when the RAN is a datapath end

$ \sum\limits_{(i,j) \in E[s} y_{i,j}^{g,b,s} \leq z_{s,g,j} \hspace{30 pt}
\forall b \in B_s,\forall s \in S, \forall g \in F_s,  \forall j \in N_c$


In [67]:
model.del_component( 'Special_VNF_link')
model.del_component( 'Special_VNF_link_index')
model.del_component( 'Special_VNF_link_index_index_0')

def Special_VNF_link_rule(model, s,g,m):
    n = Nr.index(users_sourceBS[s])+1+len(Nc)
    return sum(model.ŷ[s,g,n,m,h] for h in model.Paths_nmh[n,m]) <=  model.z[s,g,m]

model.Special_VNF_link = Constraint(model.SF, model.Nc, rule= Special_VNF_link_rule)

#### C: Anti-Affinity
The VNFs of a same type requested by a same SFCR must be mapped in different servers. In other words, two VNFs of a same time serving a same SFCR must be placed on different servers:

$\sum \limits_{\forall f\in F_s}\sum \limits_{ \forall i\in It} a_{t,i,c}^{s,f} \leq 1   \hspace{30pt} \forall s \in S \forall t\in T, \forall c\in Nc$

In [68]:
model.del_component( 'Anti_Affinity')
model.del_component( 'Anti_Affinity_index')
model.del_component( 'Anti_Affinity_index_index_0')

def Anti_Affinity_rule(model, s,c,t):
    return sum(model.a[s,f,t,i,c] for f in model.F[s] for i in model.I_t[t]) <= 1

model.Anti_Affinity = Constraint(model.S, model.Nc, model.T, rule= Anti_Affinity_rule)

#### C: I-UPF and A-UPF

The UPFs of type I-UPF  and PSA must not be co-located in the same servers

$\sum \limits_{f \in F_s} \sum \limits_{i \in I_t} z_{1,i,n}^{s,f}+ \sum \limits_{f \in F_s} \sum \limits_{i \in I_t}  z_{2,i,n}^{s,f} \leq 1  \hspace{30 pt} \forall s \in S, \forall n \in N_c$

In [69]:
model.del_component( 'IUPF_PSA')
model.del_component( 'IUPF_PSA_index')

def IUPF_PSA_rule(model, s,c):
    return sum(model.a[s,f,1,m,c] for f in model.F[s] for m in model.I_t[1])+sum(model.a[s,f,2,m,c] for f in model.F[s] for m in model.I_t[2]) <= 1

model.IUPF_PSA = Constraint(model.S, model.Nc, rule= IUPF_PSA_rule)

#### C18: Latency Requirement


$2 \cdot (\sum \limits_{f \in F_s^+} \sum \limits_{t \in T} \sum \limits_{n \in N_c} d_t  \cdot Q_{s}^{f,b}  \cdot  T_{s}^{f,t}
    +  
    \sum \limits_{ f,g \in F_s^+}  \sum\limits_{ p \in P} d_{p} \cdot O_{s}^{f,g,b}  \cdot y_{p}^{f,g,s}) + d_{\tiny DN} \leq  L_s \hspace{30 pt} \forall b \in B_s, \forall s \in S$


In [70]:
model.del_component( 'Latency')
model.del_component( 'Latency_index')

def Latency_rule(model,s,b):
    latency= 2*sum(model.z[s,f,n]*model.D_proc[t]*model.P_sbf[s,b,f]*model.T_sft[s,f,t] for t in model.T 
                   for n in model.Nc for f in model.F[s])+2*sum(model.D_prop[n,m,h]*model.y[s,f,g,n,m,h]*model.O_sbfg[s,b,f,g] 
                                                                 for n,m,h in model.Paths_cxc for f in model.F[s] 
                                                                 for g in model.F[s])+ 2*sum(model.D_prop[n,m,h]*model.ŷ[s,g,n,m,h]*model.Ô_sbg[s,b,g]
                                                                                             for n,m,h in model.Paths_rxc 
                                                                                             for g in model.F[s])+T_dn+Dict_Tproc[Ran_type] 
    return latency <= model.Lreq_s[s]

model.Latency = Constraint(model.SB, rule=Latency_rule)

#### SOLVING THE MODEL AND GETTING THE RESULTS

In [72]:
# solve the model
flag_success = False
print('Solving the model. It make take a while depending on the topology size and number os users.')

start_t_process = time.process_time()  #Process time for profiling: sum of kernel and user-space CPU

solver = SolverFactory('gurobi')

if params.FLAG_GAP:
    solver.options['mipgap'] = params.GAP_DSFC    # optimility GAP

results = solver.solve(model)

T_exec= time.process_time()  - start_t_process 



if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == 
                                                   TerminationCondition.optimal):
    print('   OK\n')
    flag_success = True
    Unassigned = 0
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('   INFEASIBLE\n')
    Unassigned = S
else:
    # Something else is wrong
    print ("   Solver Status: ",  result.solver.status,'\n')


Solving the model. It make take a while depending on the topology size and number os users.
   OK



In [73]:
# model.pprint()

In [74]:
# model.display()

In [75]:
best_cost = model.Objective.expr()
best_cost

0.15555208436724569

### Getting the results

In [76]:
# Number of VNFs
Total_upf = 0
Num_upf_type = {}
Xtic_ = [] 

for t in model.T:
    t_upf = 0
    for m in model.I_t[t]:
        for c in model.Nc:
            if model.x[t,m,c].value > 0.9:
                Total_upf += 1
                t_upf += 1
                Xtic_.append((t,m,c))
    Num_upf_type[t] = t_upf

In [77]:
# Number of open servers
Num_open_c = 0
Open_servers_list = []

for c in model.Nc:
    if model.w[c].value > 0.9:
        Open_servers_list.append(c)
        Num_open_c += 1        

In [78]:
#  nUMBER OF Relocated SESSIONS
Num_reassig = 0 # this is to guarantee similarity to the Heuriscitc. Sessions that implied the reassign
Num_reloc = 0
Lisr_rs = []
for s in model.S:
    if model.r[s].value > 0.9:
        Lisr_rs.append(s)
        Num_reloc += 1

In [81]:
# Number of migrations
Migration_lis = []
for t, m in model.U:
    for c in model.Nc:
        for e in model.Nc:
            if model.m[t,m,c,e].value > 0.9:
                Migration_lis.append((t,m,c,e))
                    
Num_mig = len(Migration_lis)                    
Num_mig

0

In [82]:
# Number of new deployments
newdeploy_list = []

for t,i in model.TI:
    if model.n[t,i].value > 0.9:
        newdeploy_list.append((t,i))

Num_newdeploy = len(newdeploy_list)

In [83]:
Latency_List = []

for s,b in model.SB:
    latency= 2*sum(model.z[s,f,n].value*model.D_proc[t]*model.P_sbf[s,b,f]*model.T_sft[s,f,t] for t in model.T 
                   for n in model.Nc for f in model.F[s])+2*sum(model.D_prop[n,m,h]*model.y[s,f,g,n,m,h].value*model.O_sbfg[s,b,f,g] 
                                                                 for n,m,h in model.Paths_cxc for f in model.F[s] 
                                                                 for g in model.F[s])+ 2*sum(model.D_prop[n,m,h]*model.ŷ[s,g,n,m,h].value*model.Ô_sbg[s,b,g]  
                                                                                             for n,m,h in model.Paths_rxc 
                                                                                             for g in model.F[s])+T_dn+Dict_Tproc[Ran_type]
    Latency_List.append(latency)


In [84]:
print('   Number of PDU sessions:', S)
print('   Execution time', T_exec)
print('   Cost', best_cost)
print('   Number of UPF installed: ', Total_upf )
print('   Number of UPFs per type', Num_upf_type )
print('   UPFs instances', Xtic_)
print('   Number of new deployments', Num_newdeploy )
print('   New UPFs list', newdeploy_list)
print('   Number of migrations', Num_mig )
print('   Migrated UPFs list', Migration_lis)
print('   Number of Open Servers', Num_open_c )
print('   Open servers list', Open_servers_list )
print('   Number of relocated sessions', Num_reloc )
print('   Reassigned sessions', Lisr_rs )
print('   Maximum latency', max(Latency_List) )
print('   Average latency', sum(Latency_List)/sum(List_B_s) )

   Number of PDU sessions: 50
   Execution time 11.859375
   Cost 0.15555208436724569
   Number of UPF installed:  4
   Number of UPFs per type {1: 2, 2: 1, 3: 1}
   UPFs instances [(1, 1, 12), (1, 2, 11), (2, 1, 12), (3, 1, 12)]
   Number of new deployments 0
   New UPFs list []
   Number of migrations 0
   Migrated UPFs list []
   Number of Open Servers 2
   Open servers list [11, 12]
   Number of relocated sessions 0
   Reassigned sessions []
   Maximum latency 972.8199999999999
   Average latency 859.0645161290324
